In [24]:
import pandas as pd
import torch
import re
from hugchat import hugchat


In [25]:
from hugchat import hugchat

def chat_with_bot(text):
    
    chatbot = hugchat.ChatBot(cookie_path="/home/ubuntu/cookies.json")
    try:
        response = chatbot.chat(text,temperature=0.2,
                                top_k=95,
                                max_new_tokens=512,
                                )
    except:
        response = "0"
    #print(text)
    # Create a new conversation
    
    id = chatbot.new_conversation()
    chatbot.change_conversation(id)
    

    # Get conversation list 
    conversation_list = chatbot.get_conversation_list()

    return conversation_list, response


In [26]:
def read_files(desired_files_df, max_sentence_length):
    for idx in range(desired_files_df.shape[0]):
        x = desired_files_df["text"].iloc[idx]
        # print("##",x)
        sentences = re.split(r'(?<=[.!?])\s+', x)
        # print("**sentences:",sentences)
        print(len(sentences))
        # break
        # Adjust the maximum sentence length as per your requirements
        chunks = sentences
        final_chunks = []
        new_data = ""
        index = 0
        index1 = -1
        for c_index,sentence in enumerate(chunks[index:index1]):
            # print("each sentence lenght:", len(sentence.split(" ")))
            if len(sentence.split(" ")) <= max_sentence_length and len((new_data+sentence).split(" "))<= max_sentence_length:
                new_data = new_data+sentence
            else:
                # print("next setence lenght:", index, len(sentence.split(" ")))
                final_chunks.append(new_data)
                new_data = ""
                index = c_index
    return final_chunks

In [28]:
path = "/home/ubuntu/cat_poc/llms/final_data.csv"
data_frame = pd.read_csv(path)

In [ ]:
data_frame

In [ ]:
data = data_frame[data_frame["filename"] == "15031-4983-FullBook.docx"]

In [ ]:
data['text']

In [ ]:
chunk_size = 400  # Adjust the chunk size as per your requirements
chunks_data = read_files(data, chunk_size)

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter_1 = TokenTextSplitter(chunk_size=400, chunk_overlap=0)
# text_splitter_2 = TokenTextSplitter(chunk_size=150, chunk_overlap=0)
chunks = text_splitter_1.split_text(data['text'][0])


In [ ]:
len(chunks)

In [ ]:
len(chunks_data)

In [ ]:
# x = chunks[5]
# print(len(x.split(" ")),x.split(" "))

In [ ]:
# x = chunks_data[5]
# print(len(x.split(" ")),x.split(" "))

In [ ]:


prompt_beginners = [
    # ''' Analyse  the following text  weather it has number_treatment give me respose in "Yes" or "No" only'''
    # ''' count no of phonetics, symbols that are there in the doccument and give me the count number '''
    '''count the number of phonetics and symbols present in the document and provide only the count just give me a number and return zero if there are none but return a number'''
    # '''Analyse  the following text  weather it has Single_or_double_quotes and give me respose in "Single" or "double" only?:''',
    # '''Analyse  the following text  weather it has Series_comma and give me respose in "Yes" or "No" only?:''',
    # '''analyse  the below text  weather it is in "American_english style" or "British english style and give me respose in "American" or "British" only?:"'''
]

Modified_chunks = []
for prompt in prompt_beginners:
    prompt_line = f'''{prompt}'''
    for chunk_i in chunks_data:
        input_text = prompt_line + chunk_i
        Modified_chunks.append(input_text)
        

In [ ]:
print(len(Modified_chunks))

In [ ]:
Modified_chunks[0]

In [ ]:
response_text = ""
count = 0

for chunk_data in Modified_chunks[:]:
    conversation_list, response = chat_with_bot(chunk_data)
    # print("Conversation List:", conversation_list)
    # print("Response:", response)
    numbers = sum(int(num) for num in re.findall(r'\d+', response))
    count += numbers
    response_text += response
    print()


In [ ]:
# numbers = re.findall(r'\d+', "There are 45 phonetic characters and 1 symbol (the bullet point character) in this document")
total = sum(int(num) for num in re.findall(r'\d+', "There are 45 phonetic characters and 1 symbol (the bullet point character) in this document"))
print("Count of numbers:", total)


In [ ]:
df = pd.DataFrame(columns=['Single_or_double_quotes', 'Series_comma','American_english',"number_treatment"])

punctuational_error_count, grammatical_error_count, spelling_error_count,missing_articles_count =[],[],[],[]
for chunk_data in Modified_chunks[:]:
    conversation_list, response = chat_with_bot(chunk_data)
    print("Conversation List:", conversation_list)
    print("Response:", response)
    print()

    # Extract the numeric value using regular expressions
    # error_count = re.findall(r'\d+', response.split("\n")[0])

    if error_count:
        # Convert the extracted value to an integer
        error_count = int(error_count[0])
    else:
        # If a numeric value is not found, try to convert words to numbers
        try:
            error_count = response.split("\n")[0]
        except ValueError:
            error_count = 0
    
    print("error_count:", response)

    # Determine the column to assign the error count based on the prompt
    if "Single_or_double_quotes" in chunk_data:
        # column_name = 'punctuational error_count'
        punctuational_error_count.append(response)
    if "Series_comma" in chunk_data:
        # column_name = 'grammatical error_count'
        grammatical_error_count.append(response)
    if "American_english " in chunk_data:
        # column_name = 'spelling_error_count'
        spelling_error_count.append(response)
    if "number_treatment" in chunk_data:
        # column_name = 'missing_articles_count'
        missing_articles_count.append(error_count)
    else:
        column_name = None

    # print("column_name: ", column_name)

    # # Add the error count to the dataframe if it is not None and column_name is valid
    # if error_count is not None and column_name:
    #     if column_name in df.columns:
    #         df[column_name] = df.get(column_name, 0) + error_count
    #     else:
    #         df[column_name] = error_count

print("punc",punctuational_error_count,len(punctuational_error_count))
print("Grammer",grammatical_error_count,len(grammatical_error_count))
print("spelling",spelling_error_count,len(spelling_error_count))
# print("arti",missing_articles_count,len(missing_articles_count))
df = pd.DataFrame({'punctuational error_count':punctuational_error_count,
                   'grammatical error_count':grammatical_error_count, 
                   'spelling_error_count':spelling_error_count})
                #    'missing_articles_count':missing_articles_count})
# Print the resulting dataframe
print(df)


In [ ]:
df

In [ ]:

# Assuming your DataFrame is named 'df'
df.rename(columns={"punctuational error_count": "Single_or_double_quotes", "grammatical error_count": "Series_comma","American_english":"Style"}, inplace=True)


In [ ]:
df.head()